<img src="https://static.itch.io/images/logo-black-new.svg" />

<center><h1> The itch.io Bundle Video Game Recommender Engine </h1></center>

<a href="https://itch.io/b/520/bundle-for-racial-justice-and-equality"> Bundle for Racial Justice and Equality </a> from <a href="https://itch.io/"> itch.io </a> was a huge hit. For a minimum donation of $5, those who donated gained an access to a library of 1741 digital items, including video games, game assets, soundtracks, comics, books, and much more.

I was determined to address the frustration that may come with getting so many choices at once: after my donation, the amount of games was so overwhelming that I did not know where to start! 

I decided to build a <b>content-based recommender engine</b> that would solve this problem for me, and other gamers who own the bundle. Since many well-known websites have already made their recommendations who they thought the best games in the bundle were, I wanted to take a different approach. Lesser-known developers also deserve the chance of a mention.

That is why this project relies solely on <b>Natural Language Processing (NLP)</b>, and recommends games based on their content. The content of the games is described mainly by <b>tags</b> (entered either by the developer on itch.io, or by users on steam), the <b>genre</b> of the game, and a <b>short decription</b> of the video game by the developer.

The itch.io Bundle Video Game Recommender uses publicly available datasets created by users <a href="https://www.reddit.com/user/theredmist"> u/theredmist </a> and <a href="https://www.reddit.com/user/azura26"> u/azura26 </a>, and are accessible on the <a href="https://www.reddit.com/r/itchioJusticeBundle/"> itchioJusticeBundle subreddit </a>. Thanks so much for this amazing resource!

In [1]:
#importing the necessary libraries
import pandas as pd
import numpy as np
import re
import contractions
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#author info
__author__ = "Lukas Barbuscak"
__email__ = "lukas.barbuscak@gmail.com"

In [2]:
#browsing the dataset
df = pd.read_csv("data.csv")
df.head()

,Title,category,steam\nreviews,steam_rating\n(5 stars),steamdb_score\n(5 stars),itch.io\nratings,itch.io\nrating,total\nratings,Net\nAvg.,Bayes\nAvg.,...,Status,Genre,Tags,Average Time,Languages,Accessibility,Multiplayer,Player\ncount,Inputs,Unnamed: 22
0,12 orbits,Game,197.0,4.0,3.99,1,NaN,198,3.980,3.785,...,Released,Action,"10-player, 2-player, 3-player, 4-player, 5-pla...",A few minutes,"German, English","Color-blind friendly, Configurable controls, O...",Local multiplayer,"""1 - 12""","Keyboard, Mouse, Xbox controller, Gamepad (any...",
1,Bleed 2,Game,367.0,4.0,4.41,37,4.7,404,4.064,3.956,...,Released,Action,"16-bit, Arcade, Boss battle, Difficult, Fast-P...",,,,,,NaN,
2,COMPEL,Game,NaN,0.0,0.00,0,NaN,0,0.000,2.500,...,Released,Action,"1-bit, 2D, Atmospheric, Black and White, Dark,...",,,,,,NaN,
3,TEOCALLI,Game,NaN,0.0,0.00,5,3.8,5,3.800,2.686,...,Released,Action,"1-bit, 2D, Seven Day Roguelike Challenge, azte...",,,,,,NaN,
4,Wondee,Game,1.0,0.0,2.97,0,NaN,0,0.000,2.500,...,Released,Action,"1d, Arcade, Colorful, Fast-Paced, gamepad, Sin...",A few minutes,English,"Color-blind friendly, Configurable controls, I...",,,"Keyboard, Gamepad (any)",


In [3]:
#removing non-text columns
df = df[["Title", "Genre", "Tags", "Short Description"]]
df = df.sort_values("Title")
df.head(15)

,Title,Genre,Tags,Short Description
340,#hasicontent,Casual,"2D, 8-Bit, bunny, PICO-8, Pixel Art, wholesome",Take fotos of the bunnys in your garden
117,// Down to Earth //,"Action, Adventure, Shooter","Fantasy, First-Person, Pixel Art, Retro, Singl...",You're a witch trying to get back to Earth aft...
189,"1,000 Heads Among the Trees",Adventure,"3D, artgame, Experimental, First-Person, Horro...",Nighttime exploration of an eccentric town in ...
383,10 in 1 game-a-week bundle!,Experimental,"artgame, bundle, Cute, Experimental, Funny, Ga...",A bunch of games! With varying levels of good-...
50,10S,Action,"Action-Adventure, Arcade, Bullet Hell, Explora...",lift the veil
439,12 Labors,"Interactive Fiction, Visual Novel","Interactive Fiction, Visual Novel",A short game about working your way up to an a...
0,12 orbits,Action,"10-player, 2-player, 3-player, 4-player, 5-pla...","12 players, one keyboard / smartphone / gamepa..."
468,1365,Platformer,"2D, artgame, Horror, memoir, mental-illness, M...","A short, autobio platformer about mental illne..."
207,2000:1: A Space Felony,Adventure,"Action-Adventure, Crime, free, Mystery, Sci-fi...",Murder mystery courtroom drama set on an inter...
170,2064: Read Only Memories,Adventure,"2064, 2064rom, Cyberpunk, Gay, LGBT, LGBTQIA, ...",A New Cyberpunk Adventure


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1071 entries, 340 to 882
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Title              1071 non-null   object
 1   Genre              1071 non-null   object
 2   Tags               1071 non-null   object
 3   Short Description  1071 non-null   object
dtypes: object(4)
memory usage: 41.8+ KB


In [5]:
#combining the text features into one
df["description"] = df["Title"] + " " + df["Genre"] + " " + df["Tags"] + " " + df["Short Description"]

#creating a series and a list with only video game titles
games_titles = df["Title"].values
games_list=games_titles.tolist()

In [6]:
#stemming
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
df['description'] = df['description'].apply(porter.stem)

#defining stopwords
stopwords = nltk.corpus.stopwords.words('english')

In [7]:
#defining a function that normalizes text features
def normalize(text):
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text, re.I|re.A)
    text = text.lower()
    text = contractions.fix(text)
    tokens = nltk.word_tokenize(text)
    filtered = [token for token in tokens if token not in stopwords]
    text = " ".join(filtered)
    return text

In [8]:
#creating the normalized text
vectorized = np.vectorize(normalize)
normalized = vectorized(list(df['description']))
print(normalized)

['hasicontent casual 2d 8bit bunny pico8 pixel art wholesome take fotos bunnys garden'
 'earth action adventure shooter fantasy firstperson pixel art retro singleplayer solodev soundtrack vaporwave witch trying get back earth crash landing mysterious planet space cats pleased'
 '1000 heads among trees adventure 3d artgame experimental firstperson horror photography travel nighttime exploration eccentric town peruvian desert'
 ...
 'used someone simulation dark humor depression firstperson queer walking simulator weird solitary walk night'
 'escape kabagahara interactive fiction adventure survival ascii experimental exploration horror retro survival horror text based unity see path kabagahara take'
 'bonsai calendar simulation bonsai colorful coop dating sim exploration minimalist perma death surreal weird piece software helps one take good care hisher bonsai tree']


In [9]:
#applying TF-IDF
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(normalized)

#computing cosine similarities
cosine_similarities = cosine_similarity(tfidf)
similarities_df = pd.DataFrame(cosine_similarities)

In [10]:
#defining the recommender function
def recommender(game, games=games_titles, similarity_score=similarities_df):
    game_id = np.where(games == game)[0][0]
    game_similarities = similarity_score.iloc[game_id].values
    similar_game_ids = np.argsort(-game_similarities)[1:6]
    similar_games = games[similar_game_ids]
    return similar_games

In [11]:
#printing the recommendations for each game
for game in games_list:
    print('Game Title:', game)
    print('Recommended:', recommender(game, games=games_titles, similarity_score=similarities_df))
    print()

Game Title: #hasicontent
Recommended: ['Best Garden' 'Galactic Wars' 'ZEPTON' 'Blaster Bunny +'
 'Pixel Session Vol.1']

Game Title: // Down to Earth //
Recommended: ['Astral Defense' 'The Aquatic Adventure of the Last Human' 'Witch Thief'
 'Space Duet™' 'They Come From Uranus!']

Game Title: 1,000 Heads Among the Trees
Recommended: ['Astæria' 'Fugue in Void' 'Quiet as a Stone'
 'Wakamarina Valley, New Zealand' 'Shutter Stroll']

Game Title: 10 in 1 game-a-week bundle!
Recommended: ['Winter Bundle (ENG)' 'Atchafalaya Arcade' 'The Night Journey' 'Astæria'
 'Jupitron Game Collection Vol.1']

Game Title: 10S
Recommended: ['The Last Librarian' 'Jet Buster' 'Lacrymo Tennis 2016 (+ 2018)'
 'Ryza Roads' 'Myth Bearer']

Game Title: 12 Labors
Recommended: ['Bunny Hill Horror' 'Serre' 'The Night Fisherman' 'Arcade Spirits'
 'a new life.']

Game Title: 12 orbits
Recommended: ['BFF or Die' 'Oh No! Bugs!' 'KeyCars' 'Turn-Based Champion' 'SHNIPERS']

Game Title: 1365
Recommended: ['Midnight Manor' '

Recommended: ['As We Know It' 'This World Unknown' 'songs and flowers' 'Belong'
 'BAKED:MAGIC']

Game Title: Boa Retina
Recommended: ['Atchafalaya Arcade' 'Fugue in Void' 'Behind the Masc'
 'Dominique Pamplemousse in "It\'s All Over Once The Fat Lady Sings!"'
 'The Space Between']

Game Title: Bold Blade
Recommended: ['The Sword and the Slime' 'Jabberwocky' 'ARGH-P-G' 'Swung' 'Dragondot 3']

Game Title: Bomb Dolls
Recommended: ['DriftKing 2D' 'Overland' 'Astral Defense' 'Keep It Together'
 'Sonar Smash']

Game Title: Bomsy
Recommended: ['Antistatic' 'Clash of Coins' 'SHNIPERS' 'Blockara' 'Turn-Based Champion']

Game Title: Bonbon
Recommended: ['Sagebrush' 'please' 'Please Follow' 'IMMURE' 'The Fall of Lazarus']

Game Title: Book of Eos
Recommended: ['Benjamin of Blackstone Edge' 'Moonstone Deep' 'Tales Across Time'
 'Legend of Hand' 'VoltAge:Genesis']

Game Title: Boom Boom Bovine
Recommended: ['B.O.O.M. - You Win [Early Access]' 'Splodey Vaders' 'Toaster Jam'
 'Peck N Run' 'Speed Farm

Game Title: Crystal Towers 2
Recommended: ['Semblance' 'LaserCat' 'Crystal Control II' 'Pichon!'
 'Throw Cubes into Brick Towers To Collapse Them']

Game Title: Cube Rampage
Recommended: ['SUPER CUBE ATTACK' 'Snake Blocks' 'Halloween Forever' 'Theorem'
 'Galactic Wars']

Game Title: Cubefall
Recommended: ['They Come From Uranus!' 'Infiniboss' "Switch 'N' Shoot" 'EVADER'
 'Over Run']

Game Title: Cuckoo Castle
Recommended: ['Tiny Dangerous Dungeons' "Me and (My) Cat's Castle -わたしと(わたしの)ねこのしろ-"
 'Aerannis' 'Vision Soft Reset' 'Gumgem']

Game Title: Curse of the Crescent Isle DX
Recommended: ['Astral Defense' 'Super Slime Arena' 'Gumgem' 'GRAVITY FIGHTERS'
 'Blitz Breaker']

Game Title: Cycle 28
Recommended: ['Cat Ace' 'Super Ledgehop: Double Laser' 'Hyper Sentinel' 'Block Out'
 'Zenodyne R']

Game Title: D.M.T
Recommended: ['Guide of the Butterfly' 'Shutter Stroll'
 'The Indifferent Wonder of an Edible Place' 'Walking Tourist'
 'Fugue in Void']

Game Title: DANGEROUS DUELS
Recommended: [

Game Title: EMUUROM
Recommended: ['Midnight Manor' 'Remnants' 'Swung' '8 Bit Space' 'Miasma Caves']

Game Title: ETHEREAL
Recommended: ['adjacency' 'stop' 'Qavo' 'Color Jumper' 'The Search']

Game Title: EVADER
Recommended: ['Retromancer' 'They Come From Uranus!' "Switch 'N' Shoot" 'Timebomb'
 'Infiniboss']

Game Title: EXTREME MEATPUNKS FOREVER
Recommended: ['Can Androids Pray: Red' 'Arcade Spirits' 'Blind Men' 'a new life.'
 '12 Labors']

Game Title: Ecchi Sketch: Draw Cute Girls Every Day! (All Ages Ver)
Recommended: ['Sweet Volley High (All Ages ver)'
 "as long as we're together: magical girls sweet & pure" 'BAKED:MAGIC'
 'songs and flowers' 'MonGirl Tile']

Game Title: Echoes of the Fey Episode 1: The Fox's Trail
Recommended: ['Blue Rose' 'Detective Bot' 'Ring of Fire Prologue'
 'Detective Hank and the Golden Sneeze' 'Daily Chthonicle']

Game Title: Eizoku
Recommended: ['IMMURE' 'please' 'Please Follow' 'Inner' 'Grime & Gaslight']

Game Title: El Interrogatorio
Recommended: ['No c

Recommended: ['Penguins Arena' 'Codemancer' 'LazerGrrl' 'Glitch Strike'
 'Soldier of Fortune']

Game Title: Glitch Strike
Recommended: ['Feud' 'Steam Marines 2' 'NEXT JUMP: Shmup Tactics' 'Turn Chase'
 'Depth of Extinction']

Game Title: Glittermitten Grove
Recommended: ['Flowerdrops' 'plant daddy' 'Secrets of Raetikon' 'The Land of Glass'
 'Even in Arcadia']

Game Title: Go Morse Go! Arcade Edition
Recommended: ['Wild Woods' 'Joggernauts' 'Speed Farmers' 'GourMelee'
 'Purple Chicken Spaceman']

Game Title: God, I Hate Wasps (1.03)
Recommended: ['Clam Man' 'The Deer God' 'The Old Man + Extra chapter' 'All You Can Eat'
 'Swung']

Game Title: Golden Treasure: The Great Green
Recommended: ['Moonstone Deep' 'Benjamin of Blackstone Edge' 'Voyageur'
 'Long Gone Days' 'Watch Me Jump']

Game Title: Golem Creation Kit
Recommended: ['Blue Rose' 'Towertale' 'Destiny Fails Us: A New Life' 'Ominous!'
 'Sorbetta: Gravely in Debt']

Game Title: Golf Peaks
Recommended: ["Brendan Keogh's Putting Challe

...

Recommended: ['Grime & Gaslight' 'Inner' 'Hollow Head' 'IMMURE' 'String Tyrant']

Game Title: NO THING
Recommended: ['ISLANDS: Non-Places' 'The Space Between' 'Dujanah'
 'Annwn: the Otherworld' 'Fumiko!']

Game Title: NOISE1
Recommended: ['Deeper Into Space' 'Pulstario' 'Space Duet™' 'MORFOSI｜莫法西'
 'Slam Fighter II']

Game Title: Nano Driller
Recommended: ['Dujanah' 'That Puzzle Game Everyone Knows' 'probability 0' 'MULTIPUCK'
 'Super Hexagon']

Game Title: Naughty Elves - Christmas puzzle game
Recommended: ['Christmas Cats Revenge' 'That Puzzle Game Everyone Knows' 'Hidden Paws'
 'Toaster Jam' 'tumbleblox']

Game Title: Nelly Cootalot: Spoonbeaks Ahoy!
Recommended: ['Panmorphia' 'Without Escape' 'The White Door'
 'Eselmir and the five magical gifts' 'Legend of Hand']

Game Title: NeoLite
Recommended: ['ACIDTRIP' 'Neon Valley: Revenge' 'Soldier of Fortune' 'Plana Gravatatis'
 'Terri-Fried']

Game Title: Neocolonialism
Recommended: ['No Pineapple Left Behind' 'Football Drama' 'Kalling K

Game Title: Pyramid
Recommended: ['[SIBERIA]' 'Inner' 'NIGHT OF THE CONSUMERS' 'String Tyrant'
 'Grime & Gaslight']

Game Title: Pyramidal Break™
Recommended: ['Bumper Boss' 'KeyCars' 'Antistatic' 'Turn-Based Champion'
 'SuperLuminauts']

Game Title: Pyre
Recommended: ['Blue Rose' 'Book of Eos' 'Towertale' 'Turn-Based Champion'
 'Ouroboros: The Sacrifice']

Game Title: QLRZ
Recommended: ['They Come From Uranus!' 'HeroRun'
 'A Lullaby of Colors (now available for Oculus Quest too)' 'Infiniboss'
 'Bubbles the Cat']

Game Title: QUACK ATTACK 1985: TURBO DX EDITION
Recommended: ['NO THING' "Switch 'N' Shoot" 'WretchWorks Arcade Pack'
 'They Come From Uranus!' 'EVADER']

Game Title: Qavo
Recommended: ['stop' 'Super Platformer Gun' 'Above: The Fallen' 'reYal' 'Perspectrum']

Game Title: Quadrilateral Cowboy
Recommended: ['nullpointer' 'INDECT' 'Eves Drop' 'BIT RAT : Singularity' 'Beglitched']

Game Title: Quench
Recommended: ['Escape from Life Inc' 'Cardinal Chains' 'Walking Tourist' 'Mendel

Recommended: ['Blitz Breaker' 'iZBOT' 'Speed Farmers' 'Gun Rounds' 'Stick Nightmare']

Game Title: SilverQuest Gaiden
Recommended: ['Emberlight' 'ARGH-P-G' 'Daydreamour' 'Roguescape'
 'Dungeon Nightmares II - The Memory']

Game Title: Six Match
Recommended: ['Garden Match' 'Monster Match' 'WaveCrash!!' 'Vegetables Deluxe C64'
 'Blockara']

Game Title: Skeletris
Recommended: ['Sverdheim' 'Self.destroy()' 'MidBoss' 'Roguescape'
 'Hyperspace Dogfights']

Game Title: Skin & Bones
Recommended: ['LaserCat' 'Pichon!' 'Adventures of a Radish(Full)'
 'Super Skull Smash GO! 2 Turbo' 'Stick Nightmare']

Game Title: Sky Rogue
Recommended: ['2d flight simulator' 'Gumgem' 'HexaCycle' 'A e r o c r a f t'
 'Rebop Blasters']

Game Title: Slam Fighter II
Recommended: ['FLAT FORM FIGHTER' 'Rebop Blasters' 'SPACEBEEF'
 '[EN/ESP] The Tail Makes the Fox - Episode 1' 'NOISE1']

Game Title: Snake Blocks
Recommended: ["A Snake's Tale" 'Super Snake 3D' 'Cube Rampage' 'SUPER CUBE ATTACK'
 'Hyperplex 3D']

Game T

Recommended: ['9 Till Void' 'Signs of the Sojourner' 'MonGirl Tile'
 'Deck Defenders [Demo Available!]' 'RogueCraft Squadron']

Game Title: The Last Librarian
Recommended: ['Myth Bearer' '10S' 'Ryza Roads'
 'The Aquatic Adventure of the Last Human' 'Silk']

Game Title: The Lost Art of Innkeeping
Recommended: ['Midnight Manor' 'Book of Eos' 'Project Kat' 'Luminous'
 'Image of Perfection']

Game Title: The Maker's Eden
Recommended: ['Ring of Fire Prologue' 'J.U.L.I.A.: Among the Stars'
 'Quadrilateral Cowboy' 'Waiting for the Loop' 'Otherside']

Game Title: The Missing Locksmith
Recommended: ['Winterlore I' 'Detective Bot' 'Escape Room Simulator - Alpha Access'
 'Veritas' 'Wooden Hearts']

Game Title: The Morrison Survival Game
Recommended: ['String Tyrant' 'Inner' '[SIBERIA]' 'Grime & Gaslight' 'MewnBase']

Game Title: The Müll Littoral
Recommended: ['Pendula Swing: The Complete Journey'
 'Eselmir and the five magical gifts' 'Blue Rose' "Old Man's Journey"
 'Ouroboros: The Sacrifice']



Recommended: ['Quiet as a Stone' "Marie's Room" 'Shutter Stroll' 'The Fall of Lazarus'
 'The Search']

Game Title: Walden, a game
Recommended: ['Peak Bleak Blues (and other moods)' 'Fugue in Void' 'The Search'
 'Guide of the Butterfly' 'Isolation Story']

Game Title: Walking Tourist
Recommended: ['Peak Bleak Blues (and other moods)' 'please' 'Please Follow' 'D.M.T'
 'soundStrider']

Game Title: Wampus
Recommended: ['Micro Mages' 'Grindhouse Games Volume I' 'iZBOT'
 'Ouroboros: The Sacrifice' 'Patches Of Adventure']

Game Title: Warden: Melody of the Undergrowth
Recommended: ['I want to be a Triangle' 'Spectres of the Cold' 'Clean ATTACK!'
 'Potato Thriller (Classic)' 'NeoLite']

Game Title: Watch Me Jump
Recommended: ['BasketBelle' 'Tales From Windy Meadow' '12 Labors' 'Super Dunkman'
 'That Which Binds Us']

Game Title: Wave to the People
Recommended: ['Darkest Wave' 'The Big Wave: Colorblind & Deaf Friendly Puzzle Game'
 'Cosmic Rochambo' 'Night in the Storm' 'Central Limit Theorem']

In [12]:
#saving the recommendations for each game into a txt file
with open("itch.io-Bundle-Recommendations.txt", "w", encoding="utf-8") as outfile:
    for game in games_list:
        results = recommender(game, games=games_titles, similarity_score=similarities_df)
        results_list = results.tolist()
        results_list.insert(0,game)
        outfile.write(str(results_list))
        outfile.write("\n")